# DSO 545 Project Data Scraping

### Scraping NBA Team Arena Location Coordinates from Wikipedia

In [1]:
# Importing packages 
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [14]:
# URL that contains a table of all 30 NBA teams and their location and arena names
wikiurl = 'https://en.wikipedia.org/wiki/National_Basketball_Association#Teams'

table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
# 200 means we are good to go! 
print(response.status_code)

200


In [15]:
# We are using BeautifulSoup package to scrape html table
soup = BeautifulSoup(response.text, 'html.parser')
# We find the table in the page 
nbatable=soup.find('table',{'class':"wikitable"})
# Convert the table into dataframe
df_nba = pd.read_html(str(nbatable))[0]

### Data Cleaning for NBA Locations Table 

In [16]:
# Fixing the table headers 
# Double headers that separate western and eastern conferences 
col_names = []
for i in df_nba.columns:
    col_names.append(i[0])
col_names

# droping row that has every value as western conference 
df_nba.columns = col_names
df_nba.drop(df_nba[df_nba['Division'] == 'Western Conference'].index, inplace = True)

# Legacy teams -> have to manually input teams that changed names or arenas 
legacy_teams = {
    'Division': ['Atlantic', 'Southwest', 'Southeast'], 
    'Team': ['New Jersey Nets', 'New Orleans Hornets', 'Charlotte Hornets'], 
    'Location': ['Newark, New Jersey', 'New Orleans, Louisiana', 'Charlotte, North Carolina'], 
    'Arena': ['Prudential Center', 'New Orleans Arena', 'Spectrum Center'], 
    'Capacity': [18711, 16867, 19077], 
    'Coordinates': ['40°44′1″N 74°10′16″W\ufeff / \ufeff40.733611°N 74.171111°W',
                    '29°56′56″N 90°04′55″W\ufeff / \ufeff29.948889°N 90.081944°W',
                    '35°13′30″N 80°50′21″W\ufeff / \ufeff35.225°N 80.839167°W'], 
    'Founded': ['1976*', '2002*', '1988*'], 
    'Joined': ['1976', '2002*', '1988*'],
    'Unnamed: 8_level_0': [np.nan, np.nan, np.nan]
}

legacy_df = pd.DataFrame(legacy_teams)
df_nba = df_nba.append(legacy_df)
df_nba.reset_index(inplace = True)
df_nba.drop(['index', 'Unnamed: 8_level_0'], axis = 1, inplace = True)


# Fixing Boston Celtics Coordinates 
# For some reason, the coordinates string didn't get parsed properly by 
df_nba.loc[0, 'Coordinates'] = df_nba.loc[0, 'Coordinates'][-49:]

# Parsing longitude and latitude from coordinates column for mapping 
df_nba['Longitude'] = df_nba['Coordinates'].map(lambda txt:  '-' + txt[txt.find('/')+3:].split(' ')[1][:-2])
df_nba['Latitude'] = df_nba['Coordinates'].map(lambda txt: txt[txt.find('/')+3:].split(' ')[0][:-2])

In [17]:
# Make a list of abbreviated team names 
team_abrv = pd.read_csv('team_abrv.txt', header = None)
team_abrv['Team_short'] = team_abrv[0].map(lambda team: team[:3])
team_abrv['Team'] = team_abrv[0].map(lambda team: team[6:])
team_abrv.drop(0, axis=1,  inplace = True)
# LAC and HOU have spaces in front of their full names so we remove the space 
team_abrv['Team'] = team_abrv['Team'].str.lstrip()

# Left join with df_nba 
locations = df_nba.merge(team_abrv, how = 'left', on='Team')

In [18]:
# Insert the correct abbreviations and add Charlotte Bobcats
locations.loc[11,'Team']='Charlotte Bobcats'
locations.loc[locations['Team']=='Charlotte Hornets','Team_short']='CHO'
locations.loc[locations['Team']=='Utah Jazz','Team_short']='UTA'

In [19]:
# Save the arena locations to csv file 
locations.to_csv('arena_locations.csv')

# All 30 current NBA teams and their locations loaded
locations

,Division,Team,Location,Arena,Capacity,Coordinates,Founded,Joined,Longitude,Latitude,Team_short
0,Atlantic,Boston Celtics,"Boston, Massachusetts",TD Garden,18624,42°21′59″N 71°03′44″W﻿ / ﻿42.366303°N 71.062228°W,1946,1946,-71.062228,42.366303,BOS
1,Atlantic,Brooklyn Nets,"New York City, New York",Barclays Center,17732,40°40′58″N 73°58′29″W﻿ / ﻿40.68265°N 73.974689°W,1967*,1976,-73.974689,40.68265,BKN
2,Atlantic,New York Knicks,"New York City, New York",Madison Square Garden,19812,40°45′02″N 73°59′37″W﻿ / ﻿40.750556°N 73.993611°W,1946,1946,-73.993611,40.750556,NYK
3,Atlantic,Philadelphia 76ers,"Philadelphia, Pennsylvania",Wells Fargo Center,21600,39°54′04″N 75°10′19″W﻿ / ﻿39.901111°N 75.171944°W,1946*,1949,-75.171944,39.901111,PHI
4,Atlantic,Toronto Raptors,"Toronto, Ontario",Scotiabank Arena,19800,43°38′36″N 79°22′45″W﻿ / ﻿43.643333°N 79.379167°W,1995,1995,-79.379167,43.643333,TOR
5,Central,Chicago Bulls,"Chicago, Illinois",United Center,20917,41°52′50″N 87°40′27″W﻿ / ﻿41.880556°N 87.674167°W,1966,1966,-87.674167,41.880556,CHI
6,Central,Cleveland Cavaliers,"Cleveland, Ohio",Rocket Mortgage FieldHouse,20562,41°29′47″N 81°41′17″W﻿ / ﻿41.496389°N 81.688056°W,1970,1970,-81.688056,41.496389,CLE
7,Central,Detroit Pistons,"Detroit, Michigan",Little Caesars Arena,20491,42°41′49″N 83°14′44″W﻿ / ﻿42.696944°N 83.245556°W,1941*,1948,-83.245556,42.696944,DET
8,Central,Indiana Pacers,"Indianapolis, Indiana",Gainbridge Fieldhouse,17923,39°45′50″N 86°09′20″W﻿ / ﻿39.763889°N 86.155556°W,1967,1976,-86.155556,39.763889,IND
9,Central,Milwaukee Bucks,"Milwaukee, Wisconsin",Fiserv Forum,18717,43°02′37″N 87°55′01″W﻿ / ﻿43.043611°N 87.916944°W,1968,1968,-87.916944,43.043611,MIL


## Scraping NBA Schedule using API 

**sportsipy**: [Click Here for link to the API](https://sportsreference.readthedocs.io/en/stable/nba.html#module-sportsipy.nba.schedule)


In [46]:
from sportsipy.nba.schedule import Schedule 

# Example look at what full schedule for a team in a season looks like from this API 
warriors_schedule = Schedule('GSW', year = '2018')
example_df = warriors_schedule.dataframe
example_df.head()

,boxscore_index,date,datetime,game,location,losses,opponent_abbr,opponent_name,playoffs,points_allowed,points_scored,result,streak,time,wins
201710170GSW,201710170GSW,"Tue, Oct 17, 2017",2017-10-17,1,Home,1,HOU,Houston Rockets,False,122,121,Loss,L 1,10:30p,0
201710200NOP,201710200NOP,"Fri, Oct 20, 2017",2017-10-20,2,Away,1,NOP,New Orleans Pelicans,False,120,128,Win,W 1,9:30p,1
201710210MEM,201710210MEM,"Sat, Oct 21, 2017",2017-10-21,3,Away,2,MEM,Memphis Grizzlies,False,111,101,Loss,L 1,8:00p,1
201710230DAL,201710230DAL,"Mon, Oct 23, 2017",2017-10-23,4,Away,2,DAL,Dallas Mavericks,False,103,133,Win,W 1,8:30p,2
201710250GSW,201710250GSW,"Wed, Oct 25, 2017",2017-10-25,5,Home,2,TOR,Toronto Raptors,False,112,117,Win,W 2,10:30p,3


In [47]:
# To get the list of all teams for our date range 
# Lakers haven't changed their name for a LONG time, so safe to use Lakers 
# Collect all unique opponent abbreviations 
lakers = pd.DataFrame(columns= example_df.columns)
# Go back to 2008-2009 season when the the OKC was first established 
for y in range(2009, 2023): 
    lakers = lakers.append(Schedule('LAL', year = y).dataframe, ignore_index=True)
team_lst = list(lakers['opponent_abbr'].unique())
# Add the Lakers to complete the list
team_lst.append('LAL')


### For Loop to scrape every team's schedule from 2008-09 season to 2021-2022 season

In [48]:
from urllib.error import HTTPError

# Empty dataframe that we will append to 
schedule = pd.DataFrame(columns= example_df.columns)
schedule

#for loop for each season and each team
for y in range(2009, 2023):
    for team in team_lst: 
        #print(y, team)
        # Error handling for if the API doesn't take the abbreviation we give
        try:
            team_df = Schedule(team, year = y).dataframe
            team_df['team'] = str(team)
            team_df['season'] = y
        except HTTPError: 
            continue
        schedule = schedule.append(team_df, ignore_index=True)

In [49]:
# Check if the scraping was done corretly 
# Normal situation: 82 games * 30 teams = 2460 games 
# 2012 -> NBA lock out due to the new Collective Bargaining Agreement 
# 2013 -> Aftermath of the Boston Marathon bombing, April 16 game scheduled in Boston between Celtics and the Pacers canceled
# 2020 -> Covid ended regular season early, some teams went to the Bubble to finish regular season 
# 2021 -> Shortened season due to Covid 
check = schedule[schedule['playoffs']==False]
check.groupby('season')['date'].count()

season
2009.0    2460
2010.0    2460
2011.0    2460
2012.0    1980
2013.0    2458
2014.0    2460
2015.0    2460
2016.0    2460
2017.0    2460
2018.0    2460
2019.0    2460
2020.0    2120
2021.0    2172
2022.0    2460
Name: date, dtype: int64

In [50]:
# Brooklyn Nets for some reason abbreviated as BRK instead of BKN from the API
# We fix this and check 
schedule.replace('BRK', 'BKN', inplace = True)
print(sum(schedule['opponent_abbr'].unique()=='BRK'))
print(sum(schedule['team'].unique()=='BRK'))


0
0


In [77]:
# change season value type from float to int 
schedule['season'] = schedule['season'].astype(int)

In [76]:
# take care of results for games that haven't been played yet 
schedule.loc[schedule['wins'].isna(), 'result'] = np.NaN

In [80]:
# matching the difference in variable name for arena locations
locations = df_nba

In [84]:
locations=locations.set_index('Team_short')

KeyError: "None of ['Team_short'] are in the columns"

In [85]:
# Add longitude and latitude columns for team and opponent in the schedule table
schedule['team_long']=schedule['team'].map(lambda team: locations.loc[team,'Longitude'])
schedule['team_lat']=schedule['team'].map(lambda team: locations.loc[team,'Latitude'])
schedule['opponent_long']=schedule['opponent_abbr'].map(lambda team: locations.loc[team,'Longitude'])
schedule['opponent_lat']=schedule['opponent_abbr'].map(lambda team: locations.loc[team,'Latitude'])

In [88]:
from geopy import distance

In [89]:
# Create a column of zeros to store the distance traveled by each team each game
distance_traveled=pd.Series([0]*len(schedule))
len(distance_traveled)

35498

In [90]:
# Insert the column of zeros into the schedule table
schedule.insert(5,"distance_traveled",distance_traveled)

In [91]:
# Add coordinates columns to the schedule table in a format that can be passed to the distance.distance function
schedule['opp_coords']=list(zip(schedule['opponent_lat'],schedule['opponent_long']))
schedule['team_coords']=list(zip(schedule['team_lat'],schedule['team_long']))

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,playoffs,...,time,wins,team,season,team_long,team_lat,opponent_long,opponent_lat,opp_coords,team_coords
0,200810280LAL,"Tue, Oct 28, 2008",2008-10-28,1,Away,0,1.0,LAL,Los Angeles Lakers,False,...,10:30p,0.0,POR,2009,-122.666667,45.531667,-118.267222,34.043056,"(34.043056, -118.267222)","(45.531667, -122.666667)"
1,200810310POR,"Fri, Oct 31, 2008",2008-10-31,2,Home,0,1.0,SAS,San Antonio Spurs,False,...,10:30p,1.0,POR,2009,-122.666667,45.531667,-98.4375,29.426944,"(29.426944, -98.4375)","(45.531667, -122.666667)"
2,200811010PHO,"Sat, Nov 1, 2008",2008-11-01,3,Away,0,2.0,PHO,Phoenix Suns,False,...,10:00p,1.0,POR,2009,-122.666667,45.531667,-112.071389,33.445833,"(33.445833, -112.071389)","(45.531667, -122.666667)"
3,200811050UTA,"Wed, Nov 5, 2008",2008-11-05,4,Away,0,3.0,UTA,Utah Jazz,False,...,9:00p,1.0,POR,2009,-122.666667,45.531667,-111.901111,40.768333,"(40.768333, -111.901111)","(45.531667, -122.666667)"
4,200811060POR,"Thu, Nov 6, 2008",2008-11-06,5,Home,0,3.0,HOU,Houston Rockets,False,...,10:30p,2.0,POR,2009,-122.666667,45.531667,-95.362222,29.750833,"(29.750833, -95.362222)","(45.531667, -122.666667)"


In [92]:
# Convert column from str to datetime
schedule['datetime']=pd.to_datetime(schedule['datetime'])

In [97]:
# sort the dataframe by team, datetime 
schedule = schedule.sort_values(by=['team', 'datetime'])
schedule.reset_index(inplace=True, drop=True)

In [99]:
# we are manually encoding the first row 
if schedule['location'][0] == 'Away': 
    schedule.loc[0,'distance_traveled']=distance.distance(schedule.loc[0,'team_coords'],schedule.loc[0,'opp_coords']).miles

schedule.head(1)

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,playoffs,...,time,wins,team,season,team_long,team_lat,opponent_long,opponent_lat,opp_coords,team_coords
0,200810290ORL,"Wed, Oct 29, 2008",2008-10-29,1,Away,401.30941,0.0,ORL,Orlando Magic,False,...,7:00p,1.0,ATL,2009,-84.396389,33.757222,-81.383611,28.539167,"(28.539167, -81.383611)","(33.757222, -84.396389)"


In [100]:
# Calculate travel distance for all remaining rows
for i,loc in enumerate(schedule['location'][1:],start=1):
    # Game is away and it is first game of season -> distance from home arena to opponent's arena
    if loc=='Away' and schedule.loc[i,'season']!=schedule.loc[i-1,'season']: 
        schedule.loc[i,'distance_traveled']=distance.distance(schedule.loc[i,'team_coords'],
                                                              schedule.loc[i,'opp_coords']).miles
    # Game is away and previous game was away -> distance from previous opponent's arena to current opponent's arena
    elif loc=='Away' and schedule.loc[i-1,'location']=='Away':
        schedule.loc[i,'distance_traveled']=distance.distance(schedule.loc[i-1,'opp_coords'],
                                                              schedule.loc[i,'opp_coords']).miles
    # Game is away and previous game was home -> distance from home arena to opponent's arena
    elif loc=='Away' and schedule.loc[i-1,'location']=='Home':
        schedule.loc[i,'distance_traveled']=distance.distance(schedule.loc[i,'team_coords'],
                                                              schedule.loc[i,'opp_coords']).miles
    # Game is home and previous game was away -> distance from previous opponent's arena to home arena
    elif loc=='Home' and schedule.loc[i-1,'location']=='Away' and schedule.loc[i,'season']==schedule.loc[i-1,'season']:
        schedule.loc[i,'distance_traveled']=distance.distance(schedule.loc[i-1,'opp_coords'],
                                                              schedule.loc[i,'team_coords']).miles
    # In all other cases the distance traveled remains zero

In [101]:
schedule.head()

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,playoffs,...,time,wins,team,season,team_long,team_lat,opponent_long,opponent_lat,opp_coords,team_coords
0,200810290ORL,"Wed, Oct 29, 2008",2008-10-29,1,Away,401.309410,0.0,ORL,Orlando Magic,False,...,7:00p,1.0,ATL,2009,-84.396389,33.757222,-81.383611,28.539167,"(28.539167, -81.383611)","(33.757222, -84.396389)"
1,200811010ATL,"Sat, Nov 1, 2008",2008-11-01,2,Home,401.309410,0.0,PHI,Philadelphia 76ers,False,...,7:00p,2.0,ATL,2009,-84.396389,33.757222,-75.171944,39.901111,"(39.901111, -75.171944)","(33.757222, -84.396389)"
2,200811050NOH,"Wed, Nov 5, 2008",2008-11-05,3,Away,424.892845,0.0,NOH,New Orleans Hornets,False,...,8:00p,3.0,ATL,2009,-84.396389,33.757222,-90.081944,29.948889,"(29.948889, -90.081944)","(33.757222, -84.396389)"
3,200811070ATL,"Fri, Nov 7, 2008",2008-11-07,4,Home,424.892845,0.0,TOR,Toronto Raptors,False,...,7:30p,4.0,ATL,2009,-84.396389,33.757222,-79.379167,43.643333,"(43.643333, -79.379167)","(33.757222, -84.396389)"
4,200811090OKC,"Sun, Nov 9, 2008",2008-11-09,5,Away,756.253175,0.0,OKC,Oklahoma City Thunder,False,...,7:00p,5.0,ATL,2009,-84.396389,33.757222,-97.515,35.463333,"(35.463333, -97.515)","(33.757222, -84.396389)"


In [102]:
# Create columns of zeros for back to back games, three games in four nights, and four games in five nights
schedule['back_to_back']=0
schedule['3_in_4']=0
schedule['4_in_5']=0

In [103]:
# Fill in columns for back to back, 3 in 4, and 4 in 5
for i,date in enumerate(schedule['datetime'][1:],start=1):
    temp1=str(date-schedule['datetime'][i-1])
    # Check for back to back
    if int(temp1[:temp1.index(' ')])==1:
        schedule.loc[i,'back_to_back']=1
    if i>=2:
        temp2=str(date-schedule['datetime'][i-2])
        # Check for 3 in 4
        if int(temp2[:temp2.index(' ')])==3:
            schedule.loc[i,'3_in_4']=1
    if i>=3:
        temp3=str(date-schedule['datetime'][i-3])
        # Check for 4 in 5
        if int(temp3[:temp3.index(' ')])==4:
            schedule.loc[i,'4_in_5']=1  

In [104]:
schedule.head()

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,playoffs,...,season,team_long,team_lat,opponent_long,opponent_lat,opp_coords,team_coords,back_to_back,3_in_4,4_in_5
0,200810290ORL,"Wed, Oct 29, 2008",2008-10-29,1,Away,401.309410,0.0,ORL,Orlando Magic,False,...,2009,-84.396389,33.757222,-81.383611,28.539167,"(28.539167, -81.383611)","(33.757222, -84.396389)",0,0,0
1,200811010ATL,"Sat, Nov 1, 2008",2008-11-01,2,Home,401.309410,0.0,PHI,Philadelphia 76ers,False,...,2009,-84.396389,33.757222,-75.171944,39.901111,"(39.901111, -75.171944)","(33.757222, -84.396389)",0,0,0
2,200811050NOH,"Wed, Nov 5, 2008",2008-11-05,3,Away,424.892845,0.0,NOH,New Orleans Hornets,False,...,2009,-84.396389,33.757222,-90.081944,29.948889,"(29.948889, -90.081944)","(33.757222, -84.396389)",0,0,0
3,200811070ATL,"Fri, Nov 7, 2008",2008-11-07,4,Home,424.892845,0.0,TOR,Toronto Raptors,False,...,2009,-84.396389,33.757222,-79.379167,43.643333,"(43.643333, -79.379167)","(33.757222, -84.396389)",0,0,0
4,200811090OKC,"Sun, Nov 9, 2008",2008-11-09,5,Away,756.253175,0.0,OKC,Oklahoma City Thunder,False,...,2009,-84.396389,33.757222,-97.515,35.463333,"(35.463333, -97.515)","(33.757222, -84.396389)",0,0,0


In [107]:
schedule['back_to_back'].sum()

7149

In [52]:
# Save the schedule table to csv
schedule.to_csv('schedule.csv')

In [11]:
schedule=pd.read_csv('schedule.csv')
schedule = schedule[schedule['playoffs']==0]

schedule.groupby('season')['game'].count()

season
2009    2460
2010    2460
2011    2460
2012    1980
2013    2458
2014    2460
2015    2460
2016    2460
2017    2460
2018    2460
2019    2460
2020    2120
2021    2172
2022    2460
Name: game, dtype: int64

In [12]:
schedule['opponent_abbr'].unique()

array(['LAL', 'SAS', 'PHO', 'UTA', 'HOU', 'MIN', 'ORL', 'MIA', 'NOH',
       'GSW', 'CHI', 'SAC', 'DET', 'NYK', 'WAS', 'BOS', 'TOR', 'LAC',
       'DEN', 'DAL', 'PHI', 'NJN', 'CHA', 'MIL', 'CLE', 'OKC', 'MEM',
       'ATL', 'IND', 'POR', 'BKN', 'NOP', 'CHO'], dtype=object)